<a href="https://colab.research.google.com/github/luisscrc/Actividad-de-Machine-Learning-Supervisado/blob/main/ML_Supervisado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# El dataset elegido es un registro de comentarios de la red social X (twitter pa la banda)
# Fuente del dataset  https://github.com/raulrrv/Deteccion_Xenofobia_TT/blob/master/data/dataset_1_clasificado.csv

Se eligió este dataset debido a la practicidad de entender los datos. Se cuentan con básicamente tres columnas. Xenofobia, Lenguaje ofensivo y Neither  (no hay mal lenguaje). Donde la Xenofobia del comentario toma valores de 0 al 7; se asigna valores de 0 a 9 según el nivel de "dureza" de las palabras. Neither toma valores del 0 al 9.

In [ ]:
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git ## LIBRERÍA PARA ACELERAR POR GPU
!python rapidsai-csp-utils/colab/pip-install.py ## COMPILAR CON T4

fatal: destination path 'rapidsai-csp-utils' already exists and is not an empty directory.
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pynvml/nvml.py", line 1798, in _LoadNvmlLibrary
    nvmlLib = CDLL("libnvidia-ml.so.1")
  File "/usr/lib/python3.10/ctypes/__init__.py", line 374, in __init__
    self._handle = _dlopen(self._name, mode)
OSError: libnvidia-ml.so.1: cannot open shared object file: No such file or directory

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/content/rapidsai-csp-utils/colab/pip-install.py", line 17, in <module>
    pynvml.nvmlInit()
  File "/usr/local/lib/python3.10/dist-packages/pynvml/nvml.py", line 1770, in nvmlInit
    nvmlInitWithFlags(0)
  File "/usr/local/lib/python3.10/dist-packages/pynvml/nvml.py", line 1753, in nvmlInitWithFlags
    _LoadNvmlLibrary()
  File "/usr/local/lib/python3.10/dist-packages/pynvml/nvml.py", line 1800, in _LoadNvmlLibrary


# CÓDIGO

In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.datasets import make_blobs #ayuda a crear el dataset

In [ ]:
import pandas as pd
import numpy as np
data=pd.read_csv('dataset_1_clasificado.csv',low_memory=False) # RAM INFINITAAAAA
data.columns

Index(['Unnamed: 0', 'count', 'xenophobia', 'offensive_language', 'neither',
       'clase', 'tweet'],
      dtype='object')

In [ ]:
df=data

df = df.drop(columns=['Unnamed: 0','neither','tweet'])
df.columns

Index(['count', 'xenophobia', 'offensive_language', 'clase'], dtype='object')

In [ ]:
for col in ['xenophobia']:
  df[col]=df[col].astype(str)

from sklearn.preprocessing import LabelEncoder

le= LabelEncoder()

for col in ['xenophobia','clase']:
  df[col]=le.fit_transform(df[col])

In [ ]:
X=df.drop(columns=['offensive_language'])
y=df['offensive_language']

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24783 entries, 0 to 24782
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype
---  ------              --------------  -----
 0   count               24783 non-null  int64
 1   xenophobia          24783 non-null  int64
 2   offensive_language  24783 non-null  int64
 3   clase               24783 non-null  int64
dtypes: int64(4)
memory usage: 774.6 KB


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
clf=RandomForestClassifier(random_state=42)
clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)

print(classification_report(y_test,y_pred))


              precision    recall  f1-score   support

           0       0.77      0.97      0.85       689
           1       0.92      0.53      0.67       428
           2       1.00      0.65      0.79       865
           3       0.90      1.00      0.95      2666
           4       0.83      0.83      0.83        42
           5       0.89      0.64      0.74        75
           6       0.85      1.00      0.92       166
           7       0.83      0.50      0.62        10
           8       0.60      0.75      0.67         8
           9       0.73      1.00      0.84         8

    accuracy                           0.88      4957
   macro avg       0.83      0.79      0.79      4957
weighted avg       0.90      0.88      0.88      4957



In [ ]:
!pip install --upgrade pyarrow

# MODELO KNN

Se elige este modelo debido a que es muy rápido con datasets grandes como este (aproximadamente 24783 registros.) No ocupa muchas clases (tres en este caso.)

In [1]:
import cuml
from cuml.ensemble import RandomForestClassifier
from cuml.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report

cuml_RFC = RandomForestClassifier(random_state=42)
cuml_KNC = KNeighborsClassifier()

classifiers = [ # lista
    {'Random Forest': cuml_RFC},
    {'KNN': cuml_KNC}
]

for classifier in classifiers:
    name, clf = list(classifier.items())[0]  # Extrae el nombre y clasificador del diccionario
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(f'Modelo {name} resultados')
    print(classification_report(y_test, y_pred))

ModuleNotFoundError: ignored